In [2]:
import os
from experiment_setup import setups
from model_tracker import track_training_C02_emissions

In [16]:
dataset = "coveo"    # coveo, diginetica, rees46, retailrocket, yoochoose
loss_function = "xe"  # bprmax, xe

dataset_path = f"../datasets/{dataset}"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [17]:
params = setups[dataset][f"params_{loss_function}"]

In [18]:
train_path = os.path.join(dataset_path,f"{dataset}_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,f"{dataset}_processed_view_test.tsv")

In [19]:
def create_recpack_script(model_name, train_path, test_path, loss, optim, layers, embedding, hidden_size, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, bpreg, n_epochs, n_sample, m, use_correct_weight_init):
    s_train_test_full = (
        f"python ../recpack/main.py --train_path {train_path} "
        f"--test_path {test_path} --m {m} --n_epochs {n_epochs} --loss {loss} --optim {optim} "
        f"--layers {layers} --embedding {embedding} --hidden_size {hidden_size} "
        f"--batch_size {batch_size} --dropout_p_embed {dropout_p_embed} "
        f"--dropout_p_hidden {dropout_p_hidden} --learning_rate {learning_rate}"
        f"{f' --n_sample {n_sample}' if loss=='bpr-max' else ''}"
        f"{' --use_correct_weight_init' if use_correct_weight_init else ''}"
        f"{f' --bpreg {bpreg}' if loss=='bpr-max' else ''}"
    )
    return s_train_test_full

In [20]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_sample = params["n_sample"]
n_epochs = 5
m = '1 5 10 20'

## Train & test (major fix model)

In [21]:
train_test_script_majorfix = create_recpack_script(model_name=f'recpack_majorfix_{loss_function}', train_path=train_path, test_path=test_path, loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, bpreg=bpreg, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=True)

In [22]:
track_training_C02_emissions(train_test_script_majorfix, f"recpack_{loss_function}", dataset)

[codecarbon WARNING @ 18:25:43] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 18:25:43] [setup] RAM Tracking...
[codecarbon INFO @ 18:25:43] [setup] GPU Tracking...
[codecarbon INFO @ 18:25:43] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:25:43] [setup] CPU Tracking...
[codecarbon WARNING @ 18:25:43] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:25:44] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 18:25:44] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon INFO @ 18:25:44] >>> Tracker's metadata:
[codecarbon INFO @ 18:25:44]   Platform system: Windows-11-10.0.22631-SP0
[codecarbon INFO @ 18:25:44]   Python version: 3.12.3
[codecarbon INFO @ 18:25:44]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 18:25:44]   Available RAM : 31.746 GB
[codecarbon INFO @ 18:25:44]   CPU count: 32
[codecarbon 

Salida de STDOUT:                        Args                                             Values
0               cuda_dev_id                                                  0
1                train_path  ../datasets/coveo\coveo_processed_view_train_f...
2                 test_path    ../datasets/coveo\coveo_processed_view_test.tsv
3                         m                                     [1, 5, 10, 20]
4                  n_epochs                                                  5
5                      loss                                      cross-entropy
6                     optim                                            adagrad
7                    layers                                                  1
8                 embedding                                                512
9               hidden_size                                                512
10               batch_size                                                 32
11          dropout_p_embed       

0.05187630505027504